In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
### Practice
# Page 247 of 
# https://prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=&EndDate=&InjuriesChkBx=yes&Submit=Search

#1.Get the Link
url = 'https://prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=&EndDate=&InjuriesChkBx=yes&Submit=Search&start=6150'

#2.Scrape the HTML structured info to local
page = requests.get(url)

# Optional: Check HTML structure in Python
# print(page.text)

In [3]:
#3.Convert to soup object
soup = BeautifulSoup(page.text, 'lxml')

#4.Define the table by set class to the 'name'(in HTML info)
table = soup.find('table', class_ = 'datatable center')

In [4]:
#5.1 Build DF: Scrape Headers/col names
headers = table.find_all('td')[0:5]
columns = []
for col in headers:
    column = col.text.replace('\xa0', '')
    columns.append(column)
    
injury = pd.DataFrame(columns = columns)

#5.2 Build DF: Scrape row data
# find all rows in table
rows = table.find_all('tr')

# for each row in table
for i in rows[1:]:
    # extract all table data from each rowhttp://localhost:8890/notebooks/Desktop/Winter%202024/PSTAT%20197B/P3%20Data/webscrape.ipynb#
    data = i.find_all('td')
    # for each table data, convert to text format
    row = [td.text for td in data]
    # get index of current row in df
    index = len(injury)
    # set to the data in text format
    injury.loc[index] = row
    # fix the formatting issue of the player names
    injury.loc[index,'Relinquished'] = injury.loc[index,'Relinquished'].replace(' • ','')

injury

,Date,Team,Acquired,Relinquished,Notes
0,2001-01-07,Bucks,,Jason Caffey,flu (DNP)
1,2001-01-07,Hornets,,Jamal Mashburn,bone bruise in left knee / left knee inflamma...
2,2001-01-07,Knicks,,Glen Rice (Sr.),plantar fasciatis in left foot (DNP)
3,2001-01-07,Raptors,,Antonio Davis,strained left groin (DNP)
4,2001-01-07,Wizards,,Jahidi White,sprained left knee (DNP)
5,2001-01-07,Wizards,,Rod Strickland,left shoulder tendinitis (DNP)
6,2001-01-08,Blazers,,Arvydas Sabonis,toothache (DNP)
7,2001-01-08,Bulls,,Marcus Fizer,back spasms (DTD)
8,2001-01-08,Cavaliers,,Chris Mihm,bruised anterior tibial bone (DNP)
9,2001-01-08,Cavaliers,,Matt Harpring,bone spur in left ankle (DNP)


In [6]:
# All Pages of 
# https://prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=&EndDate=&InjuriesChkBx=yes&Submit=Search

# link of first page
url = 'https://prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=&EndDate=&InjuriesChkBx=yes&Submit=Search'

# set up
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

# the link table
table = soup.find_all('table')[1]

# all <a> in the table, each contains a link
a = table.find_all('a')

# first page link
links = [url]

# run for loop to gather all links(last link in a is repetitive)
for i in a[:-1]:
    link = 'https://prosportstransactions.com/basketball/Search/' + i.get('href')
    links.append(link)

# run for loop to build df
for link in links:
    if link == url:
        page = requests.get(link)
        soup = BeautifulSoup(page.text, 'lxml')
        table = soup.find('table', class_ = 'datatable center')
        headers = table.find_all('td')[0:5] # first five are the col names
        columns = []
        for col in headers:
            column = col.text.replace('\xa0', '')
            columns.append(column)
        injury = pd.DataFrame(columns = columns)
        rows = table.find_all('tr')[1:]
        for i in rows:
            # extract all table data from each row
            data = i.find_all('td')
            # for each table data, convert to text format
            row = [td.text for td in data]
            # get index of current row in df
            index = len(injury)
            # set to the data in text format
            injury.loc[index] = row
            # fix the formatting issue of the player names
            injury.loc[index,'Relinquished'] = injury.loc[index,'Relinquished'].replace(' • ','')
            injury.loc[index,'Acquired'] = injury.loc[index,'Acquired'].replace(' • ','')
    else:
        page = requests.get(link)
        soup = BeautifulSoup(page.text, 'lxml')
        table = soup.find('table', class_ = 'datatable center')
        rows = table.find_all('tr')[1:]
        for i in rows:
            # extract all table data from each row
            data = i.find_all('td')
            # for each table data, convert to text format
            row = [td.text for td in data]
            # get index of current row in df
            index = len(injury)
            # set to the data in text format
            injury.loc[index] = row
            # fix the formatting issue of the player names
            injury.loc[index,'Relinquished'] = injury.loc[index,'Relinquished'].replace(' • ','')
            injury.loc[index,'Acquired'] = injury.loc[index,'Acquired'].replace(' • ','')

injury

,Date,Team,Acquired,Relinquished,Notes
0,1947-08-05,Bombers (BAA),,Jack Underman,fractured legs (in auto accident) (out indefi...
1,1948-02-15,Bullets (BAA),Harry Jeannette / Buddy Jeannette,,broken rib (DTD) (date approximate)
2,1949-11-12,Capitols,,Fred Scolari,fractured right cheekbone (out indefinitely)
3,1949-12-23,Knicks,,Vince Boryla,mumps (out ~2 weeks)
4,1950-01-13,Knicks,Vince Boryla,,returned to lineup (date approximate)
...,...,...,...,...,...
30930,2024-04-11,Mavericks,,Kyrie Irving,left hamstring injury (DTD)
30931,2024-04-11,Warriors,,Draymond Green,bruised right knee (DTD)
30932,2024-04-11,Warriors,,Gary Payton II,left calf injury (DTD)
30933,2024-04-11,Warriors,,Klay Thompson,right knee tendinitis (DTD)


In [7]:
import os
os.system('say "Web Scraping Completed."')

0

In [14]:
injury.to_csv('../data/injury_raw.csv', index=False)